In [38]:
import pandas as pd
import geopandas as gpd
import os
import constant as c
from shapely.wkt import loads
SSD_PATH = '/Volumes/T7/asset/'
fpath = os.path.join(SSD_PATH,'bdmaster/F_FAC_BUILDING_서울/F_FAC_BUILDING_11_202303.shp')

In [39]:
c.CBD_CDS

['11140', '11110', '11650', '11680', '11560']

In [40]:
gdf = gpd.read_file(fpath, encoding = 'cp949')
gdf = gdf[gdf['COL_ADM_SE'].isin(c.CBD_CDS)]
gdf = gdf[gdf['PNU'] != '1165010200103970011'] # drop pnu 1165010200103970011
BDM_COLS = ['BLD_NM', 'GRND_FLR', 'UGRND_FLR', 'PNU', 'ARCHAREA',
       'TOTALAREA', 'PLATAREA', 'HEIGHT', 'USABILITY', 'BC_RAT',
       'VL_RAT', 'BLDRGST_PK', 'USEAPR_DAY', 'REGIST_DAY', 
       'BD_MGT_SN','COL_ADM_SE', 'geometry']

gdf = gdf[BDM_COLS]
# gdf = gdf[gdf['USABILITY'] == '14000']
gdf = gdf.to_crs(epsg=4326)
gdf = gdf[gdf['GRND_FLR'] > 2]
mean_height_ratio = gdf['HEIGHT'].div(gdf['GRND_FLR']).mean() # 평균 층고: height / grnd_flr -> 2.7677396663890717
gdf['HEIGHT'].fillna(gdf['GRND_FLR'] * mean_height_ratio, inplace=True)
gdf.loc[gdf['HEIGHT'] == 0.0, 'HEIGHT'] = gdf.loc[gdf['HEIGHT'] == 0.0, 'GRND_FLR'] * mean_height_ratio
gdf.isnull().sum()

BLD_NM        38884
GRND_FLR          0
UGRND_FLR        12
PNU               0
ARCHAREA          1
TOTALAREA         0
PLATAREA         23
HEIGHT            0
USABILITY        26
BC_RAT           27
VL_RAT           29
BLDRGST_PK       58
USEAPR_DAY     1038
REGIST_DAY      632
BD_MGT_SN      3964
COL_ADM_SE        0
geometry          0
dtype: int64

In [41]:
gdf.to_csv('cbds.csv', index=False)

In [47]:
scbd = gdf[gdf['COL_ADM_SE'].isin(c.SCBD_CDS)] 
gbd = gdf[gdf['COL_ADM_SE'].isin(c.GBD_CDS)] 
ybd = gdf[gdf['COL_ADM_SE'].isin(c.YBD_CDS)] 
scbd.to_csv('scbd.csv', index=False)
gbd.to_csv('gbd.csv', index=False)
ybd.to_csv('ybd.csv', index=False)

### SCBD

In [95]:
cluster_gdf = pd.read_csv('scbd_clusters.csv')
cluster_gdf['geometry_grids'] = cluster_gdf['geometry_grids'].apply(lambda x: loads(x))
# Create a GeoDataFrame from the DataFrame
cluster_gdf = gpd.GeoDataFrame(cluster_gdf, geometry='geometry_grids')
# Set the original CRS to EPSG 5179
cluster_gdf.crs = 'epsg:5179'
# Reproject the GeoDataFrame to EPSG 4326
cluster_gdf = cluster_gdf.to_crs('epsg:4326')
cluster_gdf = cluster_gdf.drop('geometry', axis = 1)
cluster_gdf = cluster_gdf[[col for col in cluster_gdf.columns if col != 'geometry_grids'] + ['geometry_grids']]

In [96]:
cluster_gdf.to_csv('scbd_clusters_kepler.csv', index=False)

### GBD

In [89]:
cluster_gdf = pd.read_csv('gbd_clusters.csv')
cluster_gdf['geometry_grids'] = cluster_gdf['geometry_grids'].apply(lambda x: loads(x))
# Create a GeoDataFrame from the DataFrame
cluster_gdf = gpd.GeoDataFrame(cluster_gdf, geometry='geometry_grids')
# Set the original CRS to EPSG 5179
cluster_gdf.crs = 'epsg:5179'
# Reproject the GeoDataFrame to EPSG 4326
cluster_gdf = cluster_gdf.to_crs('epsg:4326')
cluster_gdf = cluster_gdf.drop('geometry', axis = 1)
cluster_gdf = cluster_gdf[[col for col in cluster_gdf.columns if col != 'geometry_grids'] + ['geometry_grids']]

In [90]:
cluster_gdf.to_csv('gbd_clusters_kepler.csv', index=False)

### YBD

In [93]:
cluster_gdf = pd.read_csv('ybd_clusters.csv')
cluster_gdf['geometry_grids'] = cluster_gdf['geometry_grids'].apply(lambda x: loads(x))
# Create a GeoDataFrame from the DataFrame
cluster_gdf = gpd.GeoDataFrame(cluster_gdf, geometry='geometry_grids')
# Set the original CRS to EPSG 5179
cluster_gdf.crs = 'epsg:5179'
# Reproject the GeoDataFrame to EPSG 4326
cluster_gdf = cluster_gdf.to_crs('epsg:4326')
cluster_gdf = cluster_gdf.drop('geometry', axis = 1)
cluster_gdf = cluster_gdf[[col for col in cluster_gdf.columns if col != 'geometry_grids'] + ['geometry_grids']]

In [94]:
cluster_gdf.to_csv('ybd_clusters_kepler.csv', index=False)